Sentiment Analysis: A study of Lexical Method, Neural Networks and SVMs
=====================================================================

In this project, we try to estimate the sentiment associated with reviews from Amazon, Yelp, and IMDB and how they perform on different review types (restaurant, movies etc). Our first model is a lexicon based sentiment analyzer
----

In [3]:
# Making the necessary imports
import numpy as np
import pandas as pd

# NLP toolkit
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize

# Regex in heuristics
import re

# For NN
from sklearn.neural_network import MLPClassifier
# For SVM
from sklearn import svm

In [4]:
# nltk stopwords
stop = stopwords.words('english')

In [5]:
print stop

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

These stopwords contain important words like 'not' and "n't" that are informative in reviews. So we edit these stopwords.
---

In [6]:
stop_edit = [u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'only', u'own', u'same', u'so', u'than', u'too', u'very', u'can', u'will', u'just', u'should', u'now']

In [7]:
stop_edit

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u'her',
 u'hers',
 u'herself',
 u'it',
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'then',
 u'once',
 u'here',
 u'there',
 u'when',
 u'where',
 u'why',
 u'how',
 u'all',
 u'any',
 

In [8]:
# Loads the reviews and returns training, test and validation sets.
def load_reviews():
    df = pd.read_csv('data.txt', delimiter='\t', header=None)
    df.columns = ['Text', 'Sentiment']
    return np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [9]:
train, validate, test = load_reviews()

In [10]:
# Combine train and validate for the lexicon based method because there are no parameters to tune.
train_validate = train.append(validate)

In [11]:
def ltz(s):
    """
    Function that lemmatizes a given token
    """
    lemmatizer = WordNetLemmatizer()
    a = lemmatizer.lemmatize(s, pos='n')
    b = lemmatizer.lemmatize(s, pos='v')
    if len(a) <= len(b):
        return a
    return b

In [12]:
def compress_word(s):
    """
    Function that compresses stressed words like 'coooooooooooooollllllllllllll' and 'wwwwwwwwaaaaaaaaaayyyyyyy'
    """
    patterns = [r'\bw+a+y+\b', r'\bc+o+o+l+\b', r'\bn+i+c+e+\b']
    matches = ['way', 'cool', 'nice']
    for i in range(len(patterns)):
        p = patterns[i]
        m = matches[i]
        rx = re.compile(p)
        if re.match(p, s):
            return matches[i]
    return s

In [13]:
compress_word('coooooooooooooooooooooooolllllllllll')

'cool'

In [14]:
def preprocessor(text):
    """
    Function that preprocesses the input string, based on some heuristics.
    """
    regex = re.compile('\.{2,}')
    regex2 = re.compile(r'(\w+)\.(\w+)')
    sentences = sent_tokenize(text)
    total = []
    for s in sentences:
        txt = regex2.sub(r'\1. \2', s)
        tokens = word_tokenize(txt)
        tokens = [s.lower() for s in tokens]
        tokens = [unicode(x, errors='ignore') for x in tokens]
        tokens = [regex.sub('', x) for x in tokens]
        tokens = [x for x in tokens if x not in '.,;!?']
        tokens = [x for x in tokens if len(x)!=0]
        tokens = [x if x != "n't" else "not" for x in tokens]
        tokens = [x if x != "'s" else "is" for x in tokens]
        tokens = [x if x != "'d" else "would" for x in tokens]
        tokens = map(ltz, tokens)
        tokens = [x for x in tokens if x not in stop_edit]
        tokens = map(compress_word, tokens)
        total = total + tokens
        total.append("<TERM>")
    return total

In [15]:
# Batch preprocessor
def batch_preprocessor(inp):
    data_ = []
    # Assuming that the columns are 'Text' and 'Sentiment'
    for index, row in inp.iterrows():
        print row['Text']
        total = preprocessor(row['Text'])
        print total
        total = ' '.join(total)
        print total
        print '==========================================================================================='
        data_.append((total, row['Sentiment']))
    return pd.DataFrame(data_)

In [16]:
curated = batch_preprocessor(train_validate)

I cannot make calls at certain places.
[u'not', u'make', u'call', u'certain', u'place', '<TERM>']
not make call certain place <TERM>
His co-stars don't fare much better, with people like Morgan Freeman, Jonah Hill, and Ed Helms just wasted.  
[u'co-star', 'not', u'fare', u'much', u'better', u'people', u'like', u'morgan', u'freeman', u'jonah', u'hill', u'ed', u'helm', u'waste', '<TERM>']
co-star not fare much better people like morgan freeman jonah hill ed helm waste <TERM>
The guy who said he's had better dialogue with his potted plants has it right.  
[u'guy', u'say', u'better', u'dialogue', u'pot', u'plant', u'ha', u'right', '<TERM>']
guy say better dialogue pot plant ha right <TERM>
10/10  
[u'10/10', '<TERM>']
10/10 <TERM>
This is a great restaurant at the Mandalay Bay.
[u'great', u'restaurant', u'mandalay', u'bay', '<TERM>']
great restaurant mandalay bay <TERM>
The "1.2" mega pixel camera, being a part of a phone, is reasonably good7.
[u'``', u'1', u'2', u"''", u'mega', u'pixel', 

A great touch.
[u'great', u'touch', '<TERM>']
great touch <TERM>
Great Earpiece.
[u'great', u'earpiece', '<TERM>']
great earpiece <TERM>
If there was ever an indication of a writer and a director's ability to meld two highly volatile temperaments into a seamless union of creativity, then this is it!  
[u'wa', u'ever', u'indication', u'writer', u'director', u'ability', u'meld', u'two', u'highly', u'volatile', u'temperament', u'seamless', u'union', u'creativity', '<TERM>']
wa ever indication writer director ability meld two highly volatile temperament seamless union creativity <TERM>
The original Body and Soul (1947) is a masterpiece.  
[u'original', u'body', u'soul', u'(', u'1947', u')', u'masterpiece', '<TERM>']
original body soul ( 1947 ) masterpiece <TERM>
The pleather case doesn't fit.
[u'pleather', u'case', 'not', u'fit', '<TERM>']
pleather case not fit <TERM>
My father has the V265, and the battery is dying.
[u'father', u'ha', u'v265', u'battery', u'die', '<TERM>']
father ha v265 

The range is very decent, I've been able to roam around my house with the phone in the living room with no reception/sound quality issues.
[u'range', u'decent', u"'ve", u'able', u'roam', u'around', u'house', u'phone', u'live', u'room', u'no', u'reception/sound', u'quality', u'issue', '<TERM>']
range decent 've able roam around house phone live room no reception/sound quality issue <TERM>
John Garfield, Ann revere, Lilli Plmer, William Conrad, Canada Lee...and filmed by one of the greatest cinematographers to ever grace the screen..James Wong Howe.  
[u'john', u'garfield', u'ann', u'revere', u'lilli', u'plmer', u'william', u'conrad', u'canada', u'lee', u'film', u'one', u'greatest', u'cinematographer', u'ever', u'grace', u'screenjames', u'wong', u'howe', '<TERM>']
john garfield ann revere lilli plmer william conrad canada lee film one greatest cinematographer ever grace screenjames wong howe <TERM>
It's one of the most boring Horror movies from the 90's mainly because it starts slow and 

[u'really', u'enjoy', u'eat', '<TERM>']
really enjoy eat <TERM>
Bad characters, bad story and bad acting.  
[u'bad', u'character', u'bad', u'story', u'bad', u'act', '<TERM>']
bad character bad story bad act <TERM>
Kathy Bates is wonderful in her characters subtle desperation and escapism; a variation on her character in "At Play In The Fields Of The Lord".  
[u'kathy', u'bat', u'wonderful', u'character', u'subtle', u'desperation', u'escapism', u'variation', u'character', u'``', u'play', u'field', u'lord', u"''", '<TERM>']
kathy bat wonderful character subtle desperation escapism variation character `` play field lord '' <TERM>
This is essentially a communications tool that does not communicate.
[u'essentially', u'communication', u'tool', u'not', u'communicate', '<TERM>']
essentially communication tool not communicate <TERM>
Very unreliable service from T-mobile !
[u'unreliable', u'service', u't-mobile', '<TERM>']
unreliable service t-mobile <TERM>
But the duet between the astronaut and

I love this thing!
[u'love', u'thing', '<TERM>']
love thing <TERM>
It's very convenient and simple to use - gets job done & makes the car ride so much smoother.
[u'convenient', u'simple', u'use', u'-', u'get', u'job', u'&', u'make', u'car', u'ride', u'much', u'smoother', '<TERM>']
convenient simple use - get job & make car ride much smoother <TERM>
All in all a beautiful directed film from Nicola's roeg wih a sublime cast.  
[u'beautiful', u'direct', u'film', u'nicola', u'roeg', u'wih', u'sublime', u'cast', '<TERM>']
beautiful direct film nicola roeg wih sublime cast <TERM>
One of my favorite purchases ever.
[u'one', u'favorite', u'purchase', u'ever', '<TERM>']
one favorite purchase ever <TERM>
I had heard good things about this place, but it exceeding every hope I could have dreamed of.
[u'hear', u'good', u'thing', u'place', u'exceed', u'every', u'hope', u'could', u'dream', '<TERM>']
hear good thing place exceed every hope could dream <TERM>
GO AND SEE IT!  
[u'go', u'see', '<TERM>']


...

[u"'ve", u'buy', u'$', u'5', u'wire', u'headphone', u'sound', u'better', '<TERM>']
've buy $ 5 wire headphone sound better <TERM>
wont work right or atleast for me.
[u'wont', u'work', u'right', u'atleast', '<TERM>']
wont work right atleast <TERM>
I came over from Verizon because cingulair has nicer cell phones.... the first thing I noticed was the really bad service.
[u'came', u'verizon', u'cingulair', u'ha', u'nicer', u'cell', u'phone', u'first', u'thing', u'notice', u'wa', u'really', u'bad', u'service', '<TERM>']
came verizon cingulair ha nicer cell phone first thing notice wa really bad service <TERM>
Very very fun chef.
[u'fun', u'chef', '<TERM>']
fun chef <TERM>
The food was very good.
[u'food', u'wa', u'good', '<TERM>']
food wa good <TERM>
I like design and look of Jabra behing the ear headsets and 5020 is pretty comfortible to wear 24 hours a day without pain in the ear.
[u'like', u'design', u'look', u'jabra', u'behing', u'ear', u'headset', u'5020', u'pretty', u'comfortible', u'

This is by far the worst purchase I've made on Amazon.
[u'far', u'worst', u'purchase', u"'ve", u'made', u'amazon', '<TERM>']
far worst purchase 've made amazon <TERM>
Kieslowski never ceases to amaze me.  
[u'kieslowski', u'never', u'cease', u'amaze', '<TERM>']
kieslowski never cease amaze <TERM>
The burger is good beef, cooked just right.
[u'burger', u'good', u'beef', u'cook', u'right', '<TERM>']
burger good beef cook right <TERM>
My fiancé and I came in the middle of the day and we were greeted and seated right away.
[u'fianc', u'came', u'middle', u'day', u'greet', u'seat', u'right', u'away', '<TERM>']
fianc came middle day greet seat right away <TERM>
Love this headset!
[u'love', u'headset', '<TERM>']
love headset <TERM>
the charger worked for about a week and then completely stopped charging my phone.
[u'charger', u'work', u'week', u'completely', u'stop', u'charge', u'phone', '<TERM>']
charger work week completely stop charge phone <TERM>
The calls drop, the phone comes on and off 

[u'get', u'angry', u'want', u'damn', u'pho', '<TERM>']
get angry want damn pho <TERM>
So in a nutshell: 1) The restaraunt smells like a combination of a dirty fish market and a sewer.
[u'nutshell', u':', u'1', u')', u'restaraunt', u'smell', u'like', u'combination', u'dirty', u'fish', u'market', u'sewer', '<TERM>']
nutshell : 1 ) restaraunt smell like combination dirty fish market sewer <TERM>
Truly awful.
[u'truly', u'awful', '<TERM>']
truly awful <TERM>
I'll even say it again  this is torture.  
[u"'ll", u'even', u'say', u'torture', '<TERM>']
'll even say torture <TERM>
One of the worst shows of all time.  
[u'one', u'worst', u'show', u'time', '<TERM>']
one worst show time <TERM>
Worth every penny.
[u'worth', u'every', u'penny', '<TERM>']
worth every penny <TERM>
After a year the battery went completely dead on my headset.
[u'year', u'battery', u'go', u'completely', u'dead', u'headset', '<TERM>']
year battery go completely dead headset <TERM>
Schrader has made a resume full of lousy,

The service here is fair at best.
[u'service', u'fair', u'best', '<TERM>']
service fair best <TERM>
The service was extremely slow.
[u'service', u'wa', u'extremely', u'slow', '<TERM>']
service wa extremely slow <TERM>
I don't know what the big deal is about this place, but I won't be back "ya'all".
['not', u'know', u'big', u'deal', u'place', u'wo', 'not', u'back', u'``', u"ya'all", u"''", '<TERM>']
not know big deal place wo not back `` ya'all '' <TERM>
the worst phone from samsung...crap..... this crappy Samsung E715.. is the worst i have ever seeen.
[u'worst', u'phone', u'samsung', u'crap', u'crappy', u'samsung', u'e715', u'worst', u'ever', u'seeen', '<TERM>']
worst phone samsung crap crappy samsung e715 worst ever seeen <TERM>
You also cannot take pictures with it in the case because the lense is covered.
[u'also', u'not', u'take', u'picture', u'case', u'lense', u'cover', '<TERM>']
also not take picture case lense cover <TERM>
You get extra minutes so that you can carry out the call

I wasted my little money with this earpiece.
[u'waste', u'little', u'money', u'earpiece', '<TERM>']
waste little money earpiece <TERM>
All in all, I can assure you I'll be back.
[u'assure', u"'ll", u'back', '<TERM>']
assure 'll back <TERM>
Love this place, hits the spot when I want something healthy but not lacking in quantity or flavor.
[u'love', u'place', u'hit', u'spot', u'want', u'something', u'healthy', u'not', u'lack', u'quantity', u'flavor', '<TERM>']
love place hit spot want something healthy not lack quantity flavor <TERM>
However, this didn't make up for the fact that overall, this was a tremendously boring movie.  
[u'however', 'not', u'make', u'fact', u'overall', u'wa', u'tremendously', u'bore', u'movie', '<TERM>']
however not make fact overall wa tremendously bore movie <TERM>
There is nothing authentic about this place.
[u'nothing', u'authentic', u'place', '<TERM>']
nothing authentic place <TERM>
This is definitely a cult classic well worth viewing and sharing with others

In [17]:
curated.columns = ['Text', 'Sentiment']

In [18]:
curated

,Text,Sentiment
0,not make call certain place <TERM>,0
1,co-star not fare much better people like morga...,0
2,guy say better dialogue pot plant ha right <TERM>,0
3,10/10 <TERM>,1
4,great restaurant mandalay bay <TERM>,1
5,`` 1 2 '' mega pixel camera part phone reasona...,1
6,saw `` mirrormask '' last night wa unsatisfact...,0
7,food wa excellent service wa good <TERM>,1
8,cashew cream sauce wa bland vegetable undercoo...,0
9,horrible horrible protector <TERM>,0


In [19]:
def create_lexicon(data):
    """
    Function that creates a lexicon from the training data
    """
    vocab = {}
    excl = ['<TERM>', '<END>']
    for index, row in data.iterrows():
        text = row['Text']
        words = text.split()
        for word in words:
            if word not in excl:
                if word in vocab:
                    vocab[word][row['Sentiment']] += 1
                else:
                    vocab[word] = [0, 0]
                    vocab[word][row['Sentiment']] += 1
    return vocab

In [20]:
vocab = create_lexicon(curated)

In [21]:
def normalize(vocab):
    """
    Function that normalizes the vocabulary
    """
    for k in vocab.keys():
        total = np.sum(vocab[k])
        vocab[k] =  [x / (1.0*total) for x in vocab[k]]

In [22]:
normalize(vocab)

In [23]:
vocab

{u'secondly': [0.0, 1.0],
 u'uhura': [1.0, 0.0],
 u'ngage': [1.0, 0.0],
 u'magnetic': [1.0, 0.0],
 u'child': [0.0, 1.0],
 u'bear': [1.0, 0.0],
 u'yellow': [0.0, 1.0],
 u'sleek': [0.0, 1.0],
 u'four': [0.0, 1.0],
 u'catch': [0.5, 0.5],
 u'sleep': [1.0, 0.0],
 u'appetite': [1.0, 0.0],
 u'captain': [1.0, 0.0],
 u'hate': [1.0, 0.0],
 u'poorly': [1.0, 0.0],
 u'whose': [0.0, 1.0],
 u'over-hip': [1.0, 0.0],
 u'buddy': [0.0, 1.0],
 u'spew': [1.0, 0.0],
 u'sweetest': [0.0, 1.0],
 u'originality': [1.0, 0.0],
 u'brilliant': [0.0, 1.0],
 u'sorrentino': [0.0, 1.0],
 u'whatsoever': [1.0, 0.0],
 u'olde': [1.0, 0.0],
 u'lord': [0.0, 1.0],
 u'sorry': [1.0, 0.0],
 u'restock': [1.0, 0.0],
 u'worth': [0.391304347826087, 0.6086956521739131],
 u'slow-moving': [1.0, 0.0],
 u'louder': [0.0, 1.0],
 u'risk': [1.0, 0.0],
 u'appal': [1.0, 0.0],
 u'blanket': [0.0, 1.0],
 u'rise': [1.0, 0.0],
 u'relatively': [0.5, 0.5],
 u'distort': [1.0, 0.0],
 u'smack': [0.0, 1.0],
 u'every': [0.34375, 0.65625],
 u'jack': [0.0, 1

In [24]:
def predict(new_sample, vocab):
    """
    Function that predicts for a new sample, from the sum of the vocab vectors
    """
    c_sample = preprocessor(new_sample)
    curr_out = np.array([0, 0])
    for w in c_sample:
        if w in vocab:
            curr_out = curr_out + np.array(vocab[w])
    tot = np.sum(curr_out)
    curr_out = curr_out / tot
    return curr_out

In [25]:
predict('Very good product', vocab)

array([ 0.31552288,  0.68447712])

Not bad! Now let's test our estimator.
---

In [26]:
count = 0
for index, row in test.iterrows():
    text = row['Text']
    print text
    p = (np.argmax(predict(text, vocab)))
    if p!=row['Sentiment']:
        count += 1

The only place good for this film is in the garbage.  
The pizza selections are good.
Jawbone Era is awesome too!
The scripting of the subtle comedy is unmatched by any movie in recent years.  
After receiving and using the product for just 2 days it broke.
Would not go back.
Perfect for the PS3.
It also took her forever to bring us the check when we asked for it.
you can even take self portraits with the outside (exterior) display, very cool.
DELETE this film from your mind!  
He was very impressed when going from the original battery to the extended battery.
The ambience is wonderful and there is music playing.
The secondary plot line is incomprehensible and its relation to the primary plot line is mystifying.  
Sorry, I will not be getting food from here anytime soon :(
Hitchcock is a great director.  
Lame would be the best way to describe it.  
Her lines seem to have been WRITTEN by a fifteen year old, though they are trying oh so, so hard to sound like how a fifteen year old woul

The block was amazing.
No real improvement.
Very good food, great atmosphere.1
Works great, when my cat attacked the phone he scratched the protective strip instead of destroying the screen.
VERY comfortable.
The acting is like watching wooden puppets moving around and reading from a book, that's how bad it is.  
The cast of veteran actors are more than just a nostalgia trip.  
This is a great phone!.
Nice case, feels good in your hands.
Brilliant!  
Don't do it!!!!
Fans of the genre will be in heaven.  
I tried the Cape Cod ravoli, chicken,with cranberry...mmmm!
This was my first and only Vegas buffet and it did not disappoint.
Phone is sturdy as all nokia bar phones are.
I got the aluminum case for my new Palm VX and it worked really well--it has protected my handheld perfectly so far.
All I have to say is the food was amazing!!!
Hands down my favorite Italian restaurant!
The attractive set used throughout most of the film is an eye-pleasing gem.  
Nothing new there.  
The acting hel

VERY funny!  
None of the three sizes they sent with the headset would stay in my ears.
Makes it easier to keep up with my bluetooth when I'm not wearing it.
It will drive you barking mad!  
Only had this a month but it's worked flawlessly so far.
Works like a charm.. Works as advertised.
My boyfriend tried the Mediterranean Chicken Salad and fell in love.
Total brunch fail.
The salad had just the right amount of sauce to not over power the scallop, which was perfectly cooked.
Verizon's bills, however, are difficult to understand even though their pricing plans are simple.
I love the Pho and the spring rolls oh so yummy you have to try.
The worst piece of crap ever along with the Verizon customer service.
Not good for the money.
Three of the most visually appealing movies i've ever seen.  
Because both ears are occupied, background is not distracting at all.
Overall, I am psyched to have a phone which has all my appointments and contacts in and gets great reception.
But it is entertain

For those that haven't seen it, don't waste your time!  
I got food poisoning here at the buffet.
I could care less... The interior is just beautiful.
Boy was that sucker dry!!.
I work in the hospitality industry in Paradise Valley and have refrained from recommending Cibo any longer.
It's not what it says it is.
It felt too light and "tinny.".
The new characters weren't all that memorable, and I found myself forgetting who was who.  
Technically, the film is well made with impressive camera-work, solid acting and effective music from Riz Ortolani  particularly good is a recurring unaccompanied female vocal that sounds like it's coming from a distant hill.  
Will order from them again!
Unfortunately it's easy to accidentally activate them with the gentle-touch buttons if you accidentally touch the phone to your face while listening.
The lead man is charisma-free.  
Its well-designed and very sharp -- the blue is a very nice color.
the only VERY DISAPPOINTING thing was there was NO SPE

This show is made for Americans - it is too stupid and full with hatred and clichés to be admitted elsewhere.  
The only good thing was our waiter, he was very helpful and kept the bloddy mary's coming.
The burger... I got the "Gold Standard" a $17 burger and was kind of disappointed.
Excellent starter wireless headset.
It only recognizes the Phone as its storage device.
The folks at Otto always make us feel so welcome and special.
It seems completely secure, both holding on to my belt, and keeping the iPhone inside.
It was horrible!.
My sister has one also and she loves it.
It's like a bad two hour TV movie.  
This is simply the BEST bluetooth headset for sound quality!
It's dumb and pointless, and a complete waste of time.  
If you haven't choked in your own vomit by the end (by all the cheap drama and worthless dialogue) you've must have bored yourself to death with this waste of time.  
Definitely not worth the $3 I paid.
During several different 2 minute calls, I experienced 5 dro

In [27]:
print float(count)/test.shape[0]

0.206666666667


Not bad for a baseline, lets move on to word vectors and neural networks
---

In [28]:
word2vec = {}
with open('glove.6B.50d.txt', 'r') as f:
    for line in f:
        tabs = line.split(' ', 1)
        word2vec[tabs[0]] = np.array([float(x) for x in tabs[1].split(' ')])

In [29]:
word2vec

{'biennials': array([  7.53020000e-01,   8.01380000e-01,  -7.38770000e-01,
         -4.98160000e-01,  -1.04830000e+00,  -5.86180000e-01,
         -2.98530000e-01,  -1.42300000e+00,  -2.03960000e-01,
          1.54980000e+00,  -1.24250000e-02,   5.02400000e-01,
         -3.64180000e-01,  -4.30270000e-01,   7.28100000e-01,
         -5.63510000e-01,  -2.22240000e-01,   7.02190000e-01,
          1.08830000e-01,  -5.71190000e-01,  -1.41050000e-01,
         -5.45710000e-01,  -3.61990000e-02,  -2.06680000e-01,
         -7.77370000e-01,   1.19480000e+00,   6.10530000e-02,
         -1.78830000e-01,  -5.05890000e-01,  -5.20990000e-01,
         -1.10480000e+00,   7.11490000e-01,   4.37160000e-01,
         -8.73380000e-01,   5.57650000e-02,   8.47260000e-02,
         -2.53160000e-01,   1.12060000e+00,  -3.65580000e-01,
          6.80500000e-02,  -7.84590000e-04,  -1.02470000e+00,
          7.25030000e-01,   4.67220000e-01,   9.32570000e-01,
          9.60820000e-01,   1.16780000e-01,  -3.62180000e

In [30]:
def list2vec(tokens):
    out = np.zeros(50)
    for t in tokens:
        if t in word2vec:
            out += word2vec[t]
    return out

In [33]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

In [74]:
# Batch preprocessor
def batch_preprocessor_NN(inp):
    data_ = []
    # Assuming that the columns are 'Text' and 'Sentiment'
    for index, row in inp.iterrows():
#         print row['Text']
        total = preprocessor(row['Text'])
        total = normalized(list2vec(total))[0]
#         print total
#         print '==========================================================================================='
        data_.append((total, row['Sentiment']))
    return pd.DataFrame(data_)

In [46]:
train_ = batch_preprocessor_NN(train)
train_.columns = ['X', 'y']
Xtrain = train_['X']
ytrain = train_['y']

I cannot make calls at certain places.
[  8.71151328e-02   4.84892793e-02  -3.19256198e-02  -1.10704005e-02
   1.10934804e-01   7.15567088e-03  -6.20677554e-02   2.93329662e-02
  -6.04020283e-02   5.29329044e-03  -3.54581265e-02   6.55568394e-02
  -1.85276005e-02  -2.02089551e-02   1.26975557e-01   1.02373563e-01
   7.15643871e-02  -9.21362069e-02   4.01075285e-04  -1.72385590e-01
  -5.10738663e-03  -8.96096133e-04   5.82123289e-02   5.56131984e-02
   4.70366461e-02  -3.75673851e-01  -9.37314748e-02  -2.53241103e-02
   1.35119012e-01  -9.83574356e-02   7.85335218e-01   9.60421743e-02
  -1.64437705e-01  -1.08563763e-01   2.62406215e-02  -3.72407887e-02
   6.83280392e-02   1.30022917e-02  -5.30779781e-02  -3.73844306e-02
  -1.99946418e-02   1.06493166e-02   2.98925746e-02   1.10021545e-01
   7.81911625e-03   6.10014733e-02  -4.89596395e-02   6.74465905e-02
   3.61482199e-02   5.69886879e-02]
His co-stars don't fare much better, with people like Morgan Freeman, Jonah Hill, and Ed Helms ju

[ 0.12294723 -0.05499185  0.01641752 -0.09992933  0.11382184  0.08540948
  0.05009371  0.05325588 -0.13838694  0.118623   -0.00684976  0.1305202
  0.04747561 -0.02005208 -0.01948238  0.15354374  0.0967121  -0.02800791
  0.12307833 -0.091054   -0.09178856  0.02979316  0.05934376  0.05323196
  0.15999912 -0.38642291 -0.18127205  0.01641802  0.16348817 -0.202029
  0.67060647  0.12506022 -0.08067758 -0.13367872 -0.04567574  0.02146694
  0.00067627 -0.05115925 -0.12205766 -0.14359377 -0.02000327  0.01077375
 -0.04552258  0.03401436  0.06856007 -0.01343277 -0.01848057  0.00289196
  0.01212838  0.068664  ]
I consider this theft.
[ 0.05825493 -0.03054816  0.01426466  0.06326975  0.08428825  0.14130198
  0.08574384 -0.02932017  0.05938732  0.11273161 -0.05515463  0.02555912
 -0.07620392 -0.10993102  0.29210347 -0.02962374  0.0050641  -0.04625301
  0.03588746 -0.12758261  0.20689273 -0.13175372 -0.11023215  0.02263973
 -0.1025997  -0.45127797 -0.05743379 -0.10136401  0.2056097  -0.07970914
  0.4

Good Service-check!
[ -6.75267217e-02   9.89200248e-02  -1.15884249e-01  -5.71755087e-02
   1.80006741e-01  -5.98472791e-02  -1.13533167e-01   2.31275132e-02
  -6.06138951e-03   1.05684841e-01  -2.01540300e-02   1.20303676e-01
  -8.98306920e-02  -1.44015639e-02   7.25761402e-02   1.54782419e-02
   1.56006348e-01   4.21638778e-02  -1.58947573e-03  -1.45391373e-01
  -1.06743682e-01   1.17191671e-01   3.85053739e-02  -9.22178278e-03
   1.66634605e-01  -3.14027909e-01  -1.46905630e-01   2.92889043e-02
   1.79932736e-01  -7.49917395e-02   7.07829420e-01   1.57222687e-01
  -2.67632463e-02   3.11105660e-03   4.00670693e-02  -6.84736063e-03
  -2.95773341e-02   1.64296806e-01   4.99230181e-02  -1.34755526e-01
  -6.97734378e-03   3.46912698e-04  -3.35944776e-02   5.12949570e-02
   2.09225435e-02   2.68182757e-02  -1.08772178e-02   5.16270308e-02
   5.94032491e-02   1.76038934e-01]
Very disappointing.  
[-0.02511314 -0.18146151  0.20439633 -0.13615909  0.09822045 -0.22835796
 -0.01351274  0.19815

Now this dish was quite flavourful.
[ 0.03344493 -0.01498799 -0.09836074  0.13736256  0.09235857 -0.09001988
 -0.09633334 -0.00952694 -0.17656342  0.07457593 -0.01433268  0.0738649
  0.25689712  0.06495265 -0.1069495   0.12228633  0.02747867  0.13741796
  0.06581788  0.05548984 -0.10348773 -0.00943224  0.23264321  0.12035332
  0.14576868 -0.19747048 -0.24402067  0.16520854  0.1384922  -0.10624155
  0.58008595 -0.01039392 -0.07590133  0.14088383  0.05296585  0.02974092
 -0.03646982  0.20249384 -0.04151452 -0.14561273  0.08783488  0.03048488
 -0.10891472 -0.03896488  0.05451205  0.15019139 -0.07143315  0.05917753
  0.14944478  0.10694744]
I just got bored watching Jessice Lange take her clothes off!  
[ 0.11279847 -0.05393882  0.12202167 -0.13634077  0.22044346  0.08673732
 -0.12784539  0.04794007  0.030363   -0.07555799 -0.02537649  0.05754716
 -0.05035531 -0.00492073  0.15989946  0.04376705  0.07624364 -0.01654866
 -0.0608952  -0.21884084 -0.01316403  0.14811685  0.12237238 -0.01888855

Battery is terrible.
[ 0.07386586 -0.01623084  0.09241066  0.00528156  0.01625404  0.07663798
  0.2836056   0.18692275  0.05565974  0.01672979 -0.06855443  0.0546277
 -0.13492197 -0.1224973   0.04113511 -0.06213775 -0.11893714  0.17474316
 -0.13233506 -0.07263127  0.01634471 -0.02686516 -0.02108838 -0.09943763
 -0.00756033 -0.36478911 -0.192719    0.01328191  0.41459126  0.01265668
  0.46915362  0.01374459  0.04885731 -0.12598278  0.09217332  0.21841345
  0.18131317 -0.07565621  0.08217422 -0.06340648 -0.02278099 -0.04382203
  0.00481354  0.0122648   0.11173417  0.06158533  0.01130716 -0.09913762
  0.07575368  0.11994652]
To summarize... the food was incredible, nay, transcendant... but nothing brings me joy quite like the memory of the pneumatic condiment dispenser.
[ 0.01121503  0.01529232 -0.05545807  0.04223594  0.02255861 -0.04989992
  0.05561614 -0.0221701   0.03634103  0.21811089 -0.0287392   0.14446368
  0.03821468 -0.08620144  0.07239772  0.11192143  0.00694839  0.08903928
  0

...

I don't know what happened in Season Five, what a mess.  
[ 0.04150782  0.00430474 -0.00197203 -0.01646584  0.07853962 -0.01766931
 -0.13935854  0.07716097 -0.09791109 -0.04083538 -0.05854849  0.07306747
 -0.18394441  0.01918236  0.27457754  0.05638738  0.07963689 -0.00188432
 -0.09402967 -0.09608157 -0.10053325  0.01853581  0.13350802  0.03256356
  0.06559231 -0.32335339 -0.12716647  0.01979952  0.17513041 -0.10320156
  0.70760223  0.14783023 -0.05759428 -0.0603806   0.04291028  0.04643564
  0.05919039  0.0434012   0.07200034 -0.07754178 -0.19098608 -0.02723874
 -0.00470635  0.09431877 -0.04393464  0.06508471 -0.01662495  0.03951358
  0.00308683  0.09265326]
Battery lasts only a few hours.
[ 0.08628628  0.03044008  0.13202729  0.06543113 -0.00273303  0.00548954
 -0.01119334  0.11076584  0.02648373  0.00229124 -0.11548199 -0.16224076
 -0.09472427 -0.01264941  0.10462978 -0.05259211 -0.19795014  0.08721747
 -0.26496836 -0.08177959  0.153889    0.03867542  0.0529285  -0.06228433
 -0.0882

[ 0.05306345  0.04322199 -0.05913643 -0.05686041  0.08530762 -0.01795831
 -0.166788   -0.08832365 -0.04268401  0.08794809 -0.01434874  0.08767111
 -0.09027084  0.01321824  0.18488288  0.02877239  0.10839671  0.04214882
 -0.01560062 -0.10680813  0.04933472  0.07473344  0.0132353  -0.00605164
  0.10086557 -0.35188746 -0.20802347 -0.03984276  0.04319394 -0.10531281
  0.75433697  0.04468749 -0.00548136 -0.17740107 -0.01528098  0.02643687
  0.01365661  0.13073731 -0.03035856 -0.06208922 -0.06819127  0.04057636
  0.02182912  0.00658669 -0.01425194  0.01292163 -0.05749663 -0.00286732
 -0.05974083  0.14064147]
It didn't charge for me.
[ 0.08360323 -0.04448999  0.04245871 -0.00850273  0.1419165   0.06556055
  0.03047382  0.02487838 -0.02744264 -0.0198453   0.0062773   0.09798204
 -0.12215652 -0.10250015  0.16336512  0.05145509  0.03834497 -0.01476019
 -0.02577715 -0.13395277  0.04655009  0.01018146  0.06256365 -0.0074049
 -0.07097037 -0.4783862   0.02630953 -0.03694379  0.07418342 -0.03135011
 

[  5.09417738e-02   1.60312572e-01  -1.61859581e-01  -1.29187258e-01
   5.60243094e-02  -1.80536652e-03  -1.21047419e-01   4.66822176e-03
  -3.10992886e-02   1.08869040e-02   1.31919366e-03   1.46136365e-01
  -6.43258575e-02  -5.46641132e-02   4.33111193e-02  -6.81823965e-02
   1.43202383e-01   7.82008209e-02  -6.03950536e-02  -8.85455919e-02
   4.68809544e-02   4.72306264e-02  -6.71583792e-02  -7.18945898e-02
   1.97210665e-01  -3.25122870e-01  -2.38526990e-01  -1.11977701e-01
   5.87491995e-02  -2.28965661e-03   7.34332307e-01   5.43116170e-02
  -1.10131541e-02  -1.23037925e-01  -2.55140337e-02   8.82297051e-02
  -4.11120448e-02   1.26140310e-01   8.86993514e-03   3.54789038e-02
  -3.25509884e-02  -6.09808041e-04  -2.93157632e-02  -7.53212299e-03
  -2.97993211e-02   5.08682412e-02  -2.53597512e-02  -4.70418265e-02
  -1.09320905e-01   1.07066811e-02]
Highly unprofessional and rude to a loyal patron!
[ 0.07712091 -0.09592272 -0.1985376  -0.13620556  0.14365241 -0.07920765
  0.04886174 

Only Pros : Large seating area/ Nice bar area/ Great simple drink menu/ The BEST brick oven pizza with homemade dough!
[-0.03336827  0.07159932 -0.06253589 -0.0736521   0.15156353  0.04374828
 -0.08286855 -0.03048877 -0.08604705  0.01336382 -0.11013118 -0.05508551
  0.05365964  0.10415203  0.01264164  0.0169604   0.0535149   0.08476255
 -0.06666072 -0.22649003  0.08576278 -0.02524454  0.05166077  0.12026837
 -0.04603372 -0.26337539 -0.16255915  0.13367364  0.21045696 -0.06723428
  0.70810208 -0.01150742 -0.14131501  0.15677193  0.03602029  0.07757883
  0.02665315  0.17175645  0.09299104 -0.03185461  0.12491918  0.0643815
 -0.08798996  0.0618578   0.04040671  0.13831225 -0.01111113 -0.11188088
  0.03134108  0.01823461]
Food was so gooodd.
[ -3.26042807e-02  -1.07605691e-01  -3.54531588e-02   1.15318341e-01
  -2.01761520e-03  -1.88657225e-01  -2.01954711e-01  -1.78614045e-01
   1.42375881e-01   2.54359646e-02   2.18488815e-02   7.46474089e-02
   2.02496188e-01  -1.46209758e-01  -9.428235

[ 0.03709818 -0.01789818  0.01749346 -0.05211658  0.09711041 -0.13322791
 -0.0458604   0.15692978 -0.03533466  0.20458277  0.02169464  0.05503435
 -0.06943909 -0.10618515  0.14861918  0.08213877  0.01416694 -0.01530477
  0.03284791 -0.1857758  -0.06524873  0.09431537  0.00452934  0.04401433
  0.2695875  -0.2108727  -0.31669989 -0.13093432  0.1501986   0.00165832
  0.48047043  0.35359206  0.09330431 -0.17910891 -0.06402698 -0.08314399
 -0.09941044  0.04530358 -0.03532078 -0.1834936  -0.0943329  -0.09641202
 -0.04481164  0.14583291  0.0575211   0.05530318  0.04478636  0.10654165
 -0.00976291  0.16041298]
I really liked that.  
[ 0.03662008  0.00441524 -0.03128084 -0.02856109  0.09316767  0.00395371
 -0.11099303 -0.05714852 -0.0763446   0.10801388 -0.05150616  0.11724448
 -0.12925021  0.04075637  0.13274932  0.08605951  0.1316668   0.0818275
  0.00650371 -0.18282335 -0.09386987  0.11453572  0.05333346  0.06935925
  0.14419051 -0.36574773 -0.2437988   0.08976265  0.18388507 -0.19278487
  0

In [84]:
train_v_ = batch_preprocessor_NN(train_validate)
train_v_.columns = ['X', 'y']
Xtrain = train_v_['X']
ytrain = train_v_['y']

In [85]:
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 50))

In [86]:
nn = clf.fit(np.array(list(Xtrain), dtype=np.float), ytrain)

valid_ = batch_preprocessor_NN(validate)

valid_.columns = ['X', 'y']

Xtest = valid_['X']

ytest = valid_['y']

y_pred = nn.predict(np.array(list(Xtest), dtype=np.float))

print y_pred

print np.dot(y_pred, 1-ytest)

count = np.sum(y_pred!=ytest)

print float(count)/len(y_pred)

[0 1 0 1 1 0 1 1 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0
 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 1
 1 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 0
 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0
 1 0 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1
 0 1 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0
 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 1 0
 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0
 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0
 1 0 0 1 1 0 1 1 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1 1
 1 0 1 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1
 1 0 0 1 1 0 1 1 0 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 

Interestingly, this is better than CNNs! :P Idk. The model probably overfit?
----

Anyway we dont have much to do here, let's try SVMs
---

In [88]:
train_ = batch_preprocessor_NN(train)
train_.columns = ['X', 'y']
Xtrain = train_['X']
ytrain = train_['y']

In [108]:
clf = svm.SVC(C=5.0)

In [105]:
clf = clf.fit(np.array(list(Xtrain), dtype=np.float), ytrain)

In [106]:
y_pred = clf.predict(np.array(list(Xtest), dtype=np.float))

print np.dot(y_pred, 1-ytest)

count = np.sum(y_pred!=ytest)

print float(count)/len(y_pred)

60
0.24


In [109]:
train_v_ = batch_preprocessor_NN(train_validate)
train_v_.columns = ['X', 'y']
Xtrain = train_v_['X']
ytrain = train_v_['y']

In [110]:
clf = clf.fit(np.array(list(Xtrain), dtype=np.float), ytrain)

In [111]:
y_pred = clf.predict(np.array(list(Xtest), dtype=np.float))

print np.dot(y_pred, 1-ytest)

count = np.sum(y_pred!=ytest)

print float(count)/len(y_pred)

57
0.238333333333


So the current rankings are:
-----
    - Neural Networks: 5% Testing Error, ~25% Training Error
    - SVMs: 23% Testing Error, 24% Training Error
    - Lexicon based methods: 20% Testing Error
    
TODO:
-----
    - K-Cross Validation
    - Plot the errors over various parameters
    - ROC?
    - Confusion Matrix